<a href="https://colab.research.google.com/github/cvjeongwon/TensorFlow2/blob/main/ngrok%2Cstreamlit%2Ch5%EC%9B%B9%EB%B0%B0%ED%8F%AC%2Cchatbot%EC%9D%91%EB%8B%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ngrok을 사용하여 image_classification.h5 모델을 생성하고 미지의 이미지 데이터로 예측을 하고자 할때 streamlit으로 쉽게 웹으로 프로토타이핑하여 포트폴리오 및 발표 자료로 사용하고자 할때 코드를 어떻게 작성하면 될까요

1. To create an image_classification.h5 model using ngrok and make predictions with unknown image data, you can follow these steps:

Train an image classification model using a dataset of labeled images and save it as image_classification.h5. Here's an example using the MobileNetV2 model:

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory='./data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    directory='./data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10, validation_data=validation_generator)
model.save('image_classification.h5')


FileNotFoundError: ignored

2. Use ngrok to create a public URL for your local server. You can run your server on your local machine and expose it to the internet using ngrok. Here's an example of how to do it:

In [ ]:
!pip install flask-ngrok

from flask_ngrok import run_with_ngrok
from flask import Flask, request

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    img = Image.open(file)
    img = img.resize((224, 224))
    img = np.asarray(img) / 255.
    img = img.reshape((1, 224, 224, 3))
    pred = model.predict(img)
    label = np.argmax(pred)
    return {'result': str(label)}

if __name__ == '__main__':
    app.run()  # Start the server


This creates a Flask server that listens for POST requests at /predict and returns the predicted label for the uploaded image.

3. To use this as a portfolio or presentation material by easily prototyping it on the web with streamlit, you can create a Streamlit app that allows users to upload an image and displays the predicted label. Here's an example:

In [ ]:
import streamlit as st
import requests

st.title('Image Classification')

uploaded_file = st.file_uploader('Choose an image', type=['jpg', 'jpeg', 'png'])
if uploaded_file is not None:
    response = requests.post('http://<ngrok-url>/predict', files={'image': uploaded_file})
    label = response.json()['result']
    st.write('Predicted label:', label)
    img = Image.open(uploaded_file)
    st.image(img, caption='Uploaded Image', use_column_width=True)
